Strictly read and follow comments for avoiding any inconvinience

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from urllib.request import Request, urlopen
import time
import random
import re

In [2]:
#the list beneath contains different user agents list to avoid getting pinned/tracked/banned while scraping
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [4]:
page_number=''

#this line generates a random number between 2 and 5 which is the amount of seconds the code stops for
sleep_var=np.random.randint(3,5)

#these are the lists the data will be stored in
property_type_storage=[]
property_price_storage=[]
property_desc_storage=[]
property_address_storage=[]
property_refrence_storage=[]


#this is the number of pages to scrap from the website, you can change it according to the preference
num=30

In [389]:
for iterator in range(num):

    url = 'https://www.homes.com/california-city-ca/homes-for-sale/'+page_number
    
     
    #Pick a random user agent
    user_agent = random.choice(user_agent_list)
        
    #this line makes sure the website doesn't block our crawler
    
    #setting headers
    req = Request(url, headers={'User-Agent': user_agent})
    
    #Make the request
    webpage = urlopen(req).read()
    
    #creating soup object
    soup1 = BeautifulSoup(webpage, 'html.parser')
    
    property_type=soup1.find_all('div',class_='property-card-content__listing-type')
    property_price=soup1.find_all('span',class_='details__price-val')
    property_desc=soup1.find_all('div',class_='property-card-content__details')
    property_address=soup1.find_all('h2',class_='property-card-content__address')
    property_refrence=soup1.find_all('a',href=True,class_='content-click')
    
    
    for i,j,k,l,m in zip(property_type,property_price,property_desc,property_address,property_refrence):
        property_type_storage.append(i.get_text())
        property_price_storage.append(j.get_text())
        property_desc_storage.append(k.get_text())
        property_address_storage.append(l.get_text())
        property_refrence_storage.append('https://www.homes.com'+m['href'])

       
    page_number='p'+str(iterator+1)
    time.sleep(sleep_var)
    
    #uncomment the line beneath to see progress
    #print("number of pages fetched -",iterator+1)

In [390]:
#To create the dataframe of the fetched data
detailed_dataframe=pd.DataFrame(list(zip(property_type_storage, property_price_storage,
                            property_desc_storage, property_address_storage, property_refrence_storage)),columns=['property_type',
                            'property_price', 'property_description','property_address','property_hyperlink'])

In [391]:
#drop duplicate entries if any
detailed_dataframe=detailed_dataframe.drop_duplicates()

In [392]:
#to construct csv file which includes all the types of properties uncomment the lines beneath

#detailed_dataframe=detailed_dataframe.replace(to_replace='-- acres',value='NaN')
#detailed_dataframe.to_csv('property_data',index=False)

In [399]:
#filtering information of residential houses/apartments whose details are present 
house_for_sale_data=detailed_dataframe[(detailed_dataframe['property_type']== 'house for sale') | (detailed_dataframe['property_type']== 'mobile manufactured for sale')]

#to construct csv file which includes only houses for sale
house_for_sale_data.to_csv('house_for_sale_data.csv',index=False)

In [400]:
#to make sure the lists are empty and there is no redundant data left, run this command at last
property_type_storage.clear()
property_price_storage.clear()
property_desc_storage.clear()
property_address_storage.clear()
property_refrence_storage.clear()


In [4]:
url = "https://www.redfin.com/state/California"


#Pick a random user agent
user_agent = random.choice(user_agent_list)

#this line makes sure the website doesn't block our crawler

#setting headers
req = Request(url, headers={'User-Agent': user_agent})

#Make the request
webpage = urlopen(req).read()

#creating soup object
soup2 = BeautifulSoup(webpage, 'html.parser')


page_number=''


#these are the lists the data will be stored in
property_type_storage=[]
property_price_storage=[]
property_desc_storage=[]
property_address_storage=[]
property_refrence_storage=[]


#this is the number of pages to scrap from the website, you can change it according to the preference till 9
num=7

In [419]:
city_href=[]
for tag in soup2.findAll('a', href=True):
    if 'city' in tag['href'] and 'https' not in tag['href']:
        city_href.append('https://www.redfin.com'+tag['href'])
        
        
for name in city_href: 
    for s in range(num):
        #for iterator in range(num):

        fetch_url=name+page_number


        #Pick a random user agent
        user_agent = random.choice(user_agent_list)

        #this line makes sure the website doesn't block our crawler

        #setting headers
        req = Request(fetch_url, headers={'User-Agent': user_agent})

        #Make the request
        webpage = urlopen(req).read()

        #creating soup object
        temp_soup = BeautifulSoup(webpage, 'html.parser')



        for i in range(20):
            property_type_storage.append('house for sale')




        spans=temp_soup.find_all('span')
        for span in spans:
            if ',' in span.get_text() and '$' in span.get_text() and ' ' not in span.get_text():
                property_price_storage.append(span.get_text())




        stats=temp_soup.find_all('div',class_='stats')

        bed=[]
        bath=[]
        area=[]
        for i in stats:
            if 'bed' in i.get_text().lower():
                bed.append(i.get_text())

            elif 'bath' in i.get_text().lower():
                bath.append(i.get_text())

            else:
                area.append(i.get_text())

        for i, j, k in zip(bed,bath,area):
            property_desc_storage.append(i+' | '+ j+' | '+ k) 




        for span in spans:
            if 'CA' in span.get_text():
                property_address_storage.append(span.get_text())




        extract=temp_soup.find_all('a',href=True,attrs={"data-rf-test-id": "slider-item-1"})
        for i in extract:
            property_refrence_storage.append('https://www.redfin.com'+i['href'])

        page_number='/page-'+ str(s+1)
        sleep_var=np.random.randint(3,5)
        time.sleep(sleep_var)

    page_number=''

    
detailed_dataframe=pd.DataFrame(list(zip(property_type_storage, property_price_storage,
                            property_desc_storage, property_address_storage, property_refrence_storage)),columns=['property_type',
                            'property_price', 'property_description','property_address','property_hyperlink'])    
#drop duplicate entries if any
detailed_dataframe=detailed_dataframe.drop_duplicates()

#filtering information of residential houses/apartments whose details are present 
house_for_sale_data=detailed_dataframe[(detailed_dataframe['property_type']== 'house for sale') | (detailed_dataframe['property_type']== 'mobile manufactured for sale')]

#to construct csv file which includes only houses for sale
house_for_sale_data.to_csv('house_for_sale_data.csv',index=False)


#to make sure the lists are empty and there is no redundant data left, run this command at last
property_type_storage.clear()
property_price_storage.clear()
property_desc_storage.clear()
property_address_storage.clear()
property_refrence_storage.clear()

HTTPError: HTTP Error 403: Forbidden

In [3]:
try:

    import requests
    from bs4 import BeautifulSoup
    import random

except:
    print(" Library Not Found !")


class Random_Proxy(object):

    def __init__(self):
        self.__url = 'https://www.sslproxies.org/'
        self.__headers = {
            'Accept-Encoding': 'gzip, deflate, sdch',
            'Accept-Language': 'en-US,en;q=0.8',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Referer': 'http://www.wikipedia.org/',
            'Connection': 'keep-alive',
            }
        self.random_ip = []
        self.random_port = []

    def __random_proxy(self):

        """
        This is Private Function Client Should not have accesss
        :return: Dictionary object of Random proxy and port number
        """

        r = requests.get(url=self.__url, headers=self.__headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        # Get the Random IP Address
        for x in soup.findAll('td')[::8]:
            self.random_ip.append(x.get_text())

        # Get Their Port
        for y in soup.findAll('td')[1::8]:
            self.random_port.append(y.get_text())

        # Zip together
        z = list(zip(self.random_ip, self.random_port))

        # This will Fetch Random IP Address and corresponding PORT Number
        number = random.randint(0, len(z)-50)
        ip_random = z[number]

        # convert Tuple into String and formart IP and PORT Address
        ip_random_string = "{}:{}".format(ip_random[0],ip_random[1])

        # Create a Proxy
        proxy = {'https':ip_random_string}

        # return Proxy
        return proxy

    def Proxy_Request(self,request_type='get',**kwargs):
        """

        :param request_type: GET, POST, PUT
        :param url: URL from which you want to do webscrapping
        :param kwargs: any other parameter you pass
        :return: Return Response
        """
        while True:
            try:
                proxy = self.__random_proxy()
                return proxy
                break
            except:
                pass